In [ ]:
import pandas as pd

In [2]:
Df_file = "../data/spacex_launches.csv"

Df = pd.read_csv(Df_file)

In [4]:
Df.shape

(205, 43)

In [5]:
Df.columns

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')

In [ ]:
# Eliminar las columnas no necesarias
#Df.drop(['window', 'net','rocket','failures','crew','ships','capsules','payloads','launchpad','date_precision','upcoming','cores','auto_update','tbd','launch_library_id','id','fairings.recovery_attempt','fairings.recovered','fairings.ships','links.flickr.small','links.presskit','fairings'], axis=1, inplace=True)

Df.drop(['success'])

In [22]:
# Renombrar las columnas en el DataFrame para que coincidan con las de la base de datos
Df.rename(columns={'fairings.reused': 'fairings_reused','links.patch.small':'links_patch_small','links.patch.large':'links_patch_large','links.reddit.campaign':'links_reddit_campaign',}, inplace=True)

Df.rename(columns={'links.reddit_launch': 'links_reddit_launch','links.reddit.media':'links_reddit_media','links.reddit.recovery':'links_reddit_recovery','links.webcast':'links_webcast'}, inplace=True)

Df.rename(columns={'links.youtube_id': 'links_youtube_id','links.article':'links_article','links.wikipedia':'links_wikipedia'}, inplace=True)

In [30]:
Df.columns

Index(['static_fire_date_utc', 'static_fire_date_unix', 'success', 'details',
       'flight_number', 'name', 'date_utc', 'date_unix', 'date_local',
       'fairings_reused', 'links_patch_small', 'links_patch_large',
       'links_reddit_campaign', 'links_reddit_launch', 'links_reddit_media',
       'links_reddit_recovery', 'links_webcast', 'links_youtube_id',
       'links_article', 'links_wikipedia'],
      dtype='object')

In [10]:
Df = Df.fillna('Null')

In [33]:
# Convertir las fechas a UTC primero y luego al formato adecuado para MySQL
Df['static_fire_date_utc'] = pd.to_datetime(Df['static_fire_date_utc'], errors='coerce', utc=True).dt.strftime('%Y-%m-%d %H:%M:%S')
Df['date_utc'] = pd.to_datetime(Df['date_utc'], errors='coerce', utc=True).dt.strftime('%Y-%m-%d %H:%M:%S')
Df['date_local'] = pd.to_datetime(Df['date_local'], errors='coerce', utc=True).dt.strftime('%Y-%m-%d %H:%M:%S')

# Reemplazar los valores 'NaT' con None para que se guarden como NULL en la base de datos
Df['static_fire_date_utc'].replace('NaT', None, inplace=True)
Df['date_utc'].replace('NaT', None, inplace=True)
Df['date_local'].replace('NaT', None, inplace=True)



C:\Users\Kike\AppData\Local\Temp\ipykernel_13088\2091846091.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Df['static_fire_date_utc'].replace('NaT', None, inplace=True)
C:\Users\Kike\AppData\Local\Temp\ipykernel_13088\2091846091.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop

In [46]:
# Using pandas
Df['success'] = Df['success'].replace({'Null': None, 'True': True, 'False': False})

# SQLAlchemy method
from sqlalchemy import func
Df['success'] = Df['success'].apply(lambda x: None if x == 'Null' else (True if x == 'True' else False))

In [48]:
# Replace 'Null' with None
Df['static_fire_date_unix'] = Df['static_fire_date_unix'].replace('Null', None)

# Alternatively, convert to float, using NaN for 'Null'
import numpy as np
Df['static_fire_date_unix'] = pd.to_numeric(Df['static_fire_date_unix'], errors='coerce')

In [49]:
# Guardar el archivo en la carpeta 'data' con el nombre 'spacex_launches_clean.csv'
output_path = '../data/spacex_launches_clean.csv'
Df.to_csv(output_path, index=False)

In [53]:
# Mostrar el DataFrame para verificar los cambios
Df.head(2)

,static_fire_date_utc,static_fire_date_unix,success,details,flight_number,name,date_utc,date_unix,date_local,fairings_reused,links_patch_small,links_patch_large,links_reddit_campaign,links_reddit_launch,links_reddit_media,links_reddit_recovery,links_webcast,links_youtube_id,links_article,links_wikipedia
0,2006-03-17 00:00:00,1.142554e+09,False,Engine failure at 33 seconds and loss of vehicle,1,FalconSat,2006-03-24 22:30:00,1143239400,2006-03-24 22:30:00,0.0,https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,Null,Null,Null,Null,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat
1,NaN,NaN,False,Successful first stage burn and transition to ...,2,DemoSat,2007-03-21 01:10:00,1174439400,2007-03-21 01:10:00,0.0,https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,Null,Null,Null,Null,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat


In [54]:
Df.shape

(205, 20)